In [1]:
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm

from keras.layers.core import Dense, Activation, Dropout
from keras.utils import np_utils, generic_utils
from keras.models import Sequential
from keras import optimizers

from sklearn.model_selection import train_test_split, cross_validate, KFold
from tensorflow.python.keras.callbacks import TensorBoard

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperas.utils import eval_hyperopt_space

In [2]:
def data():
    
    df = pd.read_csv("IEEE30_updated.csv")
    fpb = df.values
    n_rows = fpb.shape[0]
    nRVs = fpb.shape[1] - 1
    X = fpb[0:n_rows,0:nRVs]
    Y = fpb[0:n_rows,nRVs]
    total_list_X = np.reshape(X, [n_rows,nRVs])
    total_list_Y = np.reshape(Y, [n_rows,])
    X_train, X_test, Y_train, Y_test = train_test_split(total_list_X, total_list_Y, test_size=0.2, random_state=1234)
    
    return X_train, Y_train, X_test, Y_test

In [3]:
def model(X_train, Y_train, X_test, Y_test):
    
    nRVs = X_train.shape[1]
    choiceval_layer = {{choice(['four','five','six','seven'])}}
    
    model = Sequential()
    model.add(Dense({{choice([32,64,128,256])}}, input_shape=(nRVs,)))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, .5)}}))
    
    model.add(Dense({{choice([16,32,64,128])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, .5)}}))
    
    model.add(Dense({{choice([8,16,32,64])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, .5)}}))
    
    model.add(Dense({{choice([4,8,16,32])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, .5)}}))
       
    
    if choiceval_layer == 'five':
        model.add(Dense({{choice([2,4,8,16])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, .5)}}))
        
    elif choiceval_layer == 'six':
        model.add(Dense({{choice([2,4,8,16,32])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, .5)}}))
        
        model.add(Dense({{choice([2,4,8,16])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, .5)}}))
        
    elif choiceval_layer == 'seven':
        model.add(Dense({{choice([2,4,8,16,32])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, .5)}}))
        
        model.add(Dense({{choice([2,4,8,16])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, .5)}}))
        
        model.add(Dense({{choice([2,4,8,16])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, .5)}}))
        
    model.add(Dense(1))
    model.add(Activation('relu'))
    adam = optimizers.Adam(lr={{choice([10**-3, 10**-2])}})
    optim = adam
        
    model.compile(loss='mse', metrics=['accuracy'],optimizer=optim)
    
    kf = KFold(n_splits=5)
    kf.get_n_splits(total_list_X)
    accs = []
    scores = []
    
    for train_index, validate_index in kf.split(X_train):
        X_train_CV = X_train[train_index]
        X_val_CV = X_train[validate_index]
        Y_train_CV = Y_train[train_index]
        Y_val_CV = Y_train[validate_index]  
        model.fit(X_train_CV, Y_train_CV,
                  batch_size=64,
                  epochs=50,
                  verbose=0,
                  validation_data=(X_val_CV, Y_val_CV))
        score, acc = model.evaluate(X_test, Y_test, verbose=0)
        scores.append(score)
        accs.append(acc)
        
    avg_acc = np.mean(accs)
    avg_scores = np.mean(scores)
    print('Test accuracy:', avg_acc)
    return {'loss': -avg_acc, 'status': STATUS_OK, 'model': model}

In [4]:
def OptimModel():
    best_run, best_model, space = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=50,
                                          trials=Trials(),
                                          return_space=True,
                                        notebook_name = 'DNN_OptArch')
    return best_run, best_model, space

In [5]:
def Extract_BestArc(real_param_values):
    Best_Arc = Sequential()
    Best_Arc.add(Dense(real_param_values['Dense'], input_shape=(nRVs,)))
    Best_Arc.add(Activation(real_param_values['Activation']))
    Best_Arc.add(Dropout(real_param_values['Dropout']))

    Best_Arc.add(Dense(real_param_values['Dense_1']))
    Best_Arc.add(Activation(real_param_values['Activation_1']))
    Best_Arc.add(Dropout(real_param_values['Dropout_1']))

    Best_Arc.add(Dense(real_param_values['Dense_2']))
    Best_Arc.add(Activation(real_param_values['Activation_2']))
    Best_Arc.add(Dropout(real_param_values['Dropout_2']))

    Best_Arc.add(Dense(real_param_values['Dense_3']))
    Best_Arc.add(Activation(real_param_values['Activation_3']))
    Best_Arc.add(Dropout(real_param_values['Dropout_3']))


    if real_param_values['choiceval_layer'] == 'five':
        Best_Arc.add(Dense(real_param_values['Dense_4']))
        Best_Arc.add(Activation(real_param_values['Activation_4']))
        Best_Arc.add(Dropout(real_param_values['Dropout_4']))

    elif real_param_values['choiceval_layer'] == 'six':
        Best_Arc.add(Dense(real_param_values['Dense_5']))
        Best_Arc.add(Activation(real_param_values['Activation_5']))
        Best_Arc.add(Dropout(real_param_values['Dropout_5']))

        Best_Arc.add(Dense(real_param_values['Dense_6']))
        Best_Arc.add(Activation(real_param_values['Activation_6']))
        Best_Arc.add(Dropout(real_param_values['Dropout_6']))

    elif real_param_values['choiceval_layer'] == 'seven':
        Best_Arc.add(Dense(real_param_values['Dense_7']))
        Best_Arc.add(Activation(real_param_values['Activation_7']))
        Best_Arc.add(Dropout(real_param_values['Dropout_7']))

        Best_Arc.add(Dense(real_param_values['Dense_8']))
        Best_Arc.add(Activation(real_param_values['Activation_8']))
        Best_Arc.add(Dropout(real_param_values['Dropout_8']))

        Best_Arc.add(Dense(real_param_values['Dense_9']))
        Best_Arc.add(Activation(real_param_values['Activation_9']))
        Best_Arc.add(Dropout(real_param_values['Dropout_9']))

    Best_Arc.add(Dense(1))
    Best_Arc.add(Activation('relu'))
    adam = optimizers.Adam(lr=real_param_values['lr'])
    optim = adam
    Best_Arc.compile(loss='mse', metrics=['accuracy'],optimizer=optim)
    return Best_Arc

In [6]:
def epsilon_calc(predict, Pf_true, LS):
    Y_Binary = predict > LS
    Y_Binary = Y_Binary*1
    Pf = sum(Y_Binary)/len(Y_Binary)
    return (abs(Pf/Pf_true - 1))*100

In [7]:
optim_itr = 50
ITR = 60
init = 230
dj = 46

df_main = pd.read_excel("IEEE30_q_3.xlsx")
fpb_main = df_main.values
nRVs = fpb_main.shape[1] - 1
X_main = fpb_main[:,0:nRVs]
Y_main = fpb_main[:,nRVs]
df = df_main.iloc[0:init,:]
df.to_csv("IEEE30_updated.csv",index=False)

LS = 0.3
Y_Binary = Y_main > LS
Y_Binary = Y_Binary*1
Pf_True = sum(Y_Binary)/len(Y_Binary)

TrainingTime = []
Error_DNN = []
for itr in tqdm(range(0, ITR), ascii=True, unit='iterations'):
    start_time = time()
    if itr % optim_itr == 0:
        best_run, best_model, space = OptimModel()
        real_param_values = eval_hyperopt_space(space, best_run)
        end_point = init + (itr+1)*dj
        df = df_main.iloc[0:end_point,:]
        df.to_csv("IEEE30_updated.csv",index=False)
        f = open("best_run_" + str(itr) + ".txt","w")
        f.write( str(best_run) )
        f.close()
        ff = open("real_param_values_" + str(itr) + ".txt","w")
        ff.write( str(real_param_values) )
        ff.close()
        pd.DataFrame(Error_DNN).to_csv("Error_DNN.csv", index=False, header=False)
        pd.DataFrame(TrainingTime).to_csv("TrainingTime.csv", index=False, header=False)
    
    n_rows = init + itr*dj
    total_list_X = np.reshape(X_main[0:n_rows,:], [n_rows,nRVs])
    total_list_Y = np.reshape(Y_main[0:n_rows,], [n_rows,])
    X_train, X_val, Y_train, Y_val = train_test_split(total_list_X, total_list_Y, test_size=0.2, random_state=1234)
    Best_Arc_model = Extract_BestArc(real_param_values)
    Best_Arc_model.fit(X_train, Y_train,
                      batch_size=64,
                      epochs=200,
                      verbose=0,
                      validation_data=(X_val, Y_val))
    
    Y_Pred = Best_Arc_model.predict(X_main)
    epsilon = epsilon_calc(Y_Pred, Pf_True, LS)
    elapsed_time = time() - start_time
    Error_DNN.append(epsilon)
    TrainingTime.append(elapsed_time)

pd.DataFrame(Error_DNN).to_csv("Error_DNN.csv", index=False, header=False)
pd.DataFrame(TrainingTime).to_csv("TrainingTime.csv", index=False, header=False)

  0%|          | 0/60 [00:00<?, ?iterations/s]

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from time import time
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from keras.layers.core import Dense, Activation, Dropout
except:
    pass

try:
    from keras.utils import np_utils, generic_utils
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras import optimizers
except:
    pass

try:
    from sklearn.model_selection import train_test_split, cross_validate, KFold
except:
    pass

try:
    from tensorflow.python.keras.callbacks import TensorBoard
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperas.utils import eval_hyperopt_space
except:
    pass

>>> Hyperas search space:

def get_space(

0.30434781312942505                                                            

  8%|▊         | 4/50 [00:16<03:07,  4.07s/trial, best loss: -0.30434781312942505]
                                                                               Test accuracy:

  8%|▊         | 4/50 [00:20<03:07,  4.07s/trial, best loss: -0.30434781312942505]
                                                                               0.30434781312942505

 10%|█         | 5/50 [00:20<03:01,  4.04s/trial, best loss: -0.30434781312942505]
                                                                               Test accuracy:

 10%|█         | 5/50 [00:24<03:01,  4.04s/trial, best loss: -0.30434781312942505]
                                                                               0.30434781312942505

 12%|█▏        | 6/50 [00:24<02:58,  4.06s/trial, best loss: -0.30434781312942505]
                                                                               Test accuracy:

 12%|█▏        | 6/

Test accuracy:                                                                  

 42%|████▏     | 21/50 [01:30<01:56,  4.03s/trial, best loss: -0.30434781312942505]
                                                                                0.30434781312942505

 44%|████▍     | 22/50 [01:30<01:53,  4.06s/trial, best loss: -0.30434781312942505]
                                                                                Test accuracy:

 44%|████▍     | 22/50 [01:34<01:53,  4.06s/trial, best loss: -0.30434781312942505]
                                                                                0.30434781312942505

 46%|████▌     | 23/50 [01:34<01:53,  4.20s/trial, best loss: -0.30434781312942505]
                                                                                Test accuracy:

 46%|████▌     | 23/50 [01:38<01:53,  4.20s/trial, best loss: -0.30434781312942505]
                                                                                0.30434781312942505

 48

 78%|███████▊  | 39/50 [02:39<00:45,  4.10s/trial, best loss: -0.30434781312942505]
                                                                                Test accuracy:

 78%|███████▊  | 39/50 [02:43<00:45,  4.10s/trial, best loss: -0.30434781312942505]
                                                                                0.30434781312942505

 80%|████████  | 40/50 [02:43<00:41,  4.11s/trial, best loss: -0.30434781312942505]
                                                                                Test accuracy:

 80%|████████  | 40/50 [02:47<00:41,  4.11s/trial, best loss: -0.30434781312942505]
                                                                                0.30434781312942505

 82%|████████▏ | 41/50 [02:47<00:36,  4.10s/trial, best loss: -0.30434781312942505]
                                                                                Test accuracy:

 82%|████████▏ | 41/50 [02:51<00:36,  4.10s/trial, best loss: -0.30434781312942505]
      

...

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from time import time
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from keras.layers.core import Dense, Activation, Dropout
except:
    pass

try:
    from keras.utils import np_utils, generic_utils
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras import optimizers
except:
    pass

try:
    from sklearn.model_selection import train_test_split, cross_validate, KFold
except:
    pass

try:
    from tensorflow.python.keras.callbacks import TensorBoard
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperas.utils import eval_hyperopt_space
except:
    pass

>>> Hyperas search space:

def get_space(

0.3392857015132904                                                            

  8%|▊         | 4/50 [00:16<03:06,  4.06s/trial, best loss: -0.3392857015132904]
                                                                              Test accuracy:

  8%|▊         | 4/50 [00:20<03:06,  4.06s/trial, best loss: -0.3392857015132904]
                                                                              0.3392857015132904

 10%|█         | 5/50 [00:20<02:59,  4.00s/trial, best loss: -0.3392857015132904]
                                                                              Test accuracy:

 10%|█         | 5/50 [00:24<02:59,  4.00s/trial, best loss: -0.3392857015132904]
                                                                              0.3392857015132904

 12%|█▏        | 6/50 [00:24<02:56,  4.01s/trial, best loss: -0.3392857015132904]
                                                                              Test accuracy:

 12%|█▏        | 6/50 [00:28<02:

 42%|████▏     | 21/50 [01:27<01:55,  3.97s/trial, best loss: -0.3392857015132904]
                                                                               0.3392857015132904

 44%|████▍     | 22/50 [01:27<01:51,  3.99s/trial, best loss: -0.3392857015132904]
                                                                               Test accuracy:

 44%|████▍     | 22/50 [01:31<01:51,  3.99s/trial, best loss: -0.3392857015132904]
                                                                               0.3392857015132904

 46%|████▌     | 23/50 [01:31<01:48,  4.03s/trial, best loss: -0.3392857015132904]
                                                                               Test accuracy:

 46%|████▌     | 23/50 [01:35<01:48,  4.03s/trial, best loss: -0.3392857015132904]
                                                                               0.3392857015132904

 48%|████▊     | 24/50 [01:35<01:43,  3.97s/trial, best loss: -0.3392857015132904]
               

Test accuracy:                                                                 

 78%|███████▊  | 39/50 [02:39<00:43,  3.95s/trial, best loss: -0.3392857015132904]
                                                                               0.3392857015132904

 80%|████████  | 40/50 [02:39<00:40,  4.06s/trial, best loss: -0.3392857015132904]
                                                                               Test accuracy:

 80%|████████  | 40/50 [02:43<00:40,  4.06s/trial, best loss: -0.3392857015132904]
                                                                               0.3392857015132904

 82%|████████▏ | 41/50 [02:43<00:36,  4.04s/trial, best loss: -0.3392857015132904]
                                                                               Test accuracy:

 82%|████████▏ | 41/50 [02:47<00:36,  4.04s/trial, best loss: -0.3392857015132904]
                                                                               0.3392857015132904

 84%|████████▍ | 

100%|##########| 60/60 [13:01<00:00, 13.02s/iterations]
